In [43]:
import numpy as np
import sounddevice as sd
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import ttk
import threading
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# --------------------- Funciones básicas --------------------- #

def record_audio(duration=5, sample_rate=44100, channels=2):
    """
    Graba audio desde el micrófono durante el tiempo indicado.
    :param duration: Duración en segundos.
    :param sample_rate: Frecuencia de muestreo (Hz).
    :param channels: Número de canales (1 para mono).
    :return: Señal de audio grabada.
    """
    print(f"Grabando audio por {duration} segundos...")
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=channels, dtype='int16')
    sd.wait()  # Espera a que finalice la grabación
    print("Grabación completada.")
    return recording

def reduce_noise(audio_data, rate, low_cut, high_cut):
    """
    Filtra la señal en el dominio de la frecuencia, eliminando componentes
    fuera del rango definido por low_cut y high_cut.
    :param audio_data: Señal de audio (1D).
    :param rate: Frecuencia de muestreo.
    :param low_cut: Frecuencia mínima a conservar (Hz).
    :param high_cut: Frecuencia máxima a conservar (Hz).
    :return: Señal filtrada en el dominio temporal.
    """
    # Se asume que audio_data es 1D. Si no, se toma el primer canal.
    if len(audio_data.shape) > 1:
        audio_data = audio_data[:, 0]
    
    # Transformada de Fourier (parte real positiva)
    data_fft = np.fft.rfft(audio_data)
    freqs = np.fft.rfftfreq(len(audio_data), d=1.0/rate)
    
    # Crear la máscara para conservar solo las frecuencias deseadas
    mask = (freqs >= low_cut) & (freqs <= high_cut)
    data_fft_filtered = data_fft * mask
    
    # Regresar la señal filtrada al dominio temporal
    filtered_data = np.fft.irfft(data_fft_filtered)
    return filtered_data

def play_audio(audio, rate):
    """
    Reproduce el audio usando sounddevice.
    """
    sd.play(audio, rate)
    sd.wait()

# Funciones para reproducir en hilos separados (para no bloquear la interfaz)
def play_original():
    threading.Thread(target=play_audio, args=(audio_flat, sample_rate)).start()

def play_filtered():
    if filtered_audio is not None:
        threading.Thread(target=play_audio, args=(filtered_audio.astype(np.int16), sample_rate)).start()

def apply_filter():
    low = scale_low.get()
    high = scale_high.get()
    global filtered_audio
    filtered_audio = reduce_noise(audio_flat, sample_rate, low_cut=low, high_cut=high)
    wav.write(filename_filtered, sample_rate, filtered_audio.astype(np.int16))
    label_status.config(text=f"Filtro aplicado: {low} Hz - {high} Hz (guardado en {filename_filtered})")
    print(f"Filtro aplicado: {low} Hz - {high} Hz")
    update_graphs()  # Actualiza los gráficos embebidos

# Función para actualizar los gráficos en el Notebook
def update_graphs():
    # Actualizamos la gráfica de la señal en el tiempo
    ax_time.clear()
    ax_time.plot(time_axis, audio_flat, color='blue')
    ax_time.set_title("Señal Original (Tiempo)")
    ax_time.set_xlabel("Tiempo (s)")
    ax_time.set_ylabel("Amplitud")
    ax_time.grid(True)
    canvas_time.draw()
    
    # Actualizamos el espectro original
    ax_spec_orig.clear()
    ax_spec_orig.plot(freqs, np.abs(data_fft), color='green')
    ax_spec_orig.set_title("Espectro de la Señal Original")
    ax_spec_orig.set_xlabel("Frecuencia (Hz)")
    ax_spec_orig.set_ylabel("Magnitud")
    ax_spec_orig.grid(True)
    canvas_spec_orig.draw()
    
    # Actualizamos el espectro filtrado
    # Si aún no se ha aplicado un nuevo filtro, usamos el inicial
    ax_spec_filt.clear()
    if filtered_audio is not None:
        data_fft_filt = np.fft.rfft(filtered_audio)
        ax_spec_filt.plot(freqs, np.abs(data_fft_filt), color='red')
    else:
        ax_spec_filt.plot(freqs, np.abs(data_fft_filtered), color='red')
    ax_spec_filt.set_title("Espectro del Audio Filtrado")
    ax_spec_filt.set_xlabel("Frecuencia (Hz)")
    ax_spec_filt.set_ylabel("Magnitud")
    ax_spec_filt.grid(True)
    canvas_spec_filt.draw()

# --------------------- Programa principal --------------------- #

# Parámetros de grabación y nombres de archivo
sample_rate = 44100  # Hz
duration = 5         # segundos
filename_original = "audio_original.wav"
filename_filtered = "audio_filtrado.wav"

# 1. Grabar audio y guardarlo
audio = record_audio(duration=duration, sample_rate=sample_rate, channels=1)
wav.write(filename_original, sample_rate, audio)
print("Audio original guardado en:", filename_original)

# Convertir a un array 1D para el procesamiento
audio_flat = audio.flatten()

# 2. Aplicar un filtro inicial (valores por defecto, que podrás ajustar)
filtered_audio = reduce_noise(audio_flat, sample_rate, low_cut=300, high_cut=3000)
wav.write(filename_filtered, sample_rate, filtered_audio.astype(np.int16))
print("Audio filtrado guardado en:", filename_filtered)

# 3. Calcular variables para los gráficos
time_axis = np.linspace(0, len(audio_flat) / sample_rate, num=len(audio_flat))
freqs = np.fft.rfftfreq(len(audio_flat), d=1.0/sample_rate)
data_fft = np.fft.rfft(audio_flat)
data_fft_filtered = np.fft.rfft(filtered_audio)

# 4. Crear la interfaz gráfica con Tkinter y embebido de gráficos con Notebook

root = tk.Tk()
root.title("Reproducción y Filtro de Audio")
root.geometry("900x700")  # Tamaño de la ventana

# Usamos ttk para un mejor estilo
style = ttk.Style(root)
style.theme_use("clam")

# Creamos un Notebook para los gráficos
notebook = ttk.Notebook(root)
notebook.pack(fill='both', expand=True, padx=10, pady=10)

# Pestaña 1: Señal en el tiempo
frame_time = ttk.Frame(notebook)
notebook.add(frame_time, text="Tiempo")

fig_time = Figure(figsize=(5, 3), dpi=100)
ax_time = fig_time.add_subplot(111)
ax_time.plot(time_axis, audio_flat, color='blue')
ax_time.set_title("Señal Original (Tiempo)")
ax_time.set_xlabel("Tiempo (s)")
ax_time.set_ylabel("Amplitud")
ax_time.grid(True)
canvas_time = FigureCanvasTkAgg(fig_time, master=frame_time)
canvas_time.draw()
canvas_time.get_tk_widget().pack(fill='both', expand=True)

# Pestaña 2: Espectro de la señal original
frame_spec_orig = ttk.Frame(notebook)
notebook.add(frame_spec_orig, text="Espectro Original")

fig_spec_orig = Figure(figsize=(5, 3), dpi=100)
ax_spec_orig = fig_spec_orig.add_subplot(111)
ax_spec_orig.plot(freqs, np.abs(data_fft), color='green')
ax_spec_orig.set_title("Espectro de la Señal Original")
ax_spec_orig.set_xlabel("Frecuencia (Hz)")
ax_spec_orig.set_ylabel("Magnitud")
ax_spec_orig.grid(True)
canvas_spec_orig = FigureCanvasTkAgg(fig_spec_orig, master=frame_spec_orig)
canvas_spec_orig.draw()
canvas_spec_orig.get_tk_widget().pack(fill='both', expand=True)

# Pestaña 3: Espectro del audio filtrado
frame_spec_filt = ttk.Frame(notebook)
notebook.add(frame_spec_filt, text="Espectro Filtrado")

fig_spec_filt = Figure(figsize=(5, 3), dpi=100)
ax_spec_filt = fig_spec_filt.add_subplot(111)
ax_spec_filt.plot(freqs, np.abs(data_fft_filtered), color='red')
ax_spec_filt.set_title("Espectro del Audio Filtrado")
ax_spec_filt.set_xlabel("Frecuencia (Hz)")
ax_spec_filt.set_ylabel("Magnitud")
ax_spec_filt.grid(True)
canvas_spec_filt = FigureCanvasTkAgg(fig_spec_filt, master=frame_spec_filt)
canvas_spec_filt.draw()
canvas_spec_filt.get_tk_widget().pack(fill='both', expand=True)

# Marco inferior para controles y botones
frame_controls = ttk.Frame(root)
frame_controls.pack(fill='x', padx=10, pady=10)

btn_original = ttk.Button(frame_controls, text="Reproducir Audio Original", command=play_original)
btn_original.grid(row=0, column=0, padx=10, pady=5)

btn_filtered = ttk.Button(frame_controls, text="Reproducir Audio Filtrado", command=play_filtered)
btn_filtered.grid(row=0, column=1, padx=10, pady=5)

# Controles para ajustar el filtro
label_low = ttk.Label(frame_controls, text="Frecuencia mínima (Hz):")
label_low.grid(row=1, column=0, padx=10, pady=5, sticky='w')
scale_low = tk.Scale(frame_controls, from_=50, to=1000, orient=tk.HORIZONTAL)
scale_low.set(300)
scale_low.grid(row=1, column=1, padx=10, pady=5, sticky='w')

label_high = ttk.Label(frame_controls, text="Frecuencia máxima (Hz):")
label_high.grid(row=2, column=0, padx=10, pady=5, sticky='w')
scale_high = tk.Scale(frame_controls, from_=1000, to=10000, orient=tk.HORIZONTAL)
scale_high.set(3000)
scale_high.grid(row=2, column=1, padx=10, pady=5, sticky='w')

btn_apply = ttk.Button(frame_controls, text="Aplicar Filtro", command=apply_filter)
btn_apply.grid(row=3, column=0, columnspan=2, padx=10, pady=10)

label_status = ttk.Label(frame_controls, text="Ajusta los parámetros y aplica el filtro")
label_status.grid(row=4, column=0, columnspan=2, padx=10, pady=5)

root.mainloop()


Grabando audio por 5 segundos...
Grabación completada.
Audio original guardado en: audio_original.wav
Audio filtrado guardado en: audio_filtrado.wav
Filtro aplicado: 50 Hz - 5103 Hz
Filtro aplicado: 50 Hz - 10000 Hz
Filtro aplicado: 50 Hz - 1000 Hz
Filtro aplicado: 1000 Hz - 1000 Hz
Filtro aplicado: 1000 Hz - 4838 Hz
Filtro aplicado: 106 Hz - 5103 Hz
Filtro aplicado: 106 Hz - 5103 Hz
Filtro aplicado: 106 Hz - 5103 Hz
